In [1]:
!pip install keras

In [ ]:
!pip install tensorflow

In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
import seaborn as sns
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from sklearn import metrics
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import pandas as pd


ModuleNotFoundError: ignored

In [ ]:
def load_file(filepath):
 dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
 return dataframe.values

In [ ]:
def load_group(filenames, prefix=''):
 loaded = list()
 for name in filenames:
    data = load_file(prefix + name)

    loaded.append(data)
 # stack group so that features are the 3rd dimension
 loaded = dstack(loaded)
 return loaded

In [ ]:
def load_dataset_group(group, prefix=''):
 filepath = prefix + group + '/Inertial Signals/'
 print('File Path : ',filepath)
 # load all 9 files as a single array
 filenames = list()
 # total acceleration
 filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
 # body acceleration
 filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
 # body gyroscope
 filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
 # load input data
 X = load_group(filenames, filepath)
 # load class output
 y = load_file(prefix + group + '/y_'+group+'.txt')
 return X, y

In [ ]:
# load dataset
from numpy import dstack
from pandas import read_csv

# load a single file as a numpy array
def load_file(filepath):
 dataframe = read_csv(filepath, header=None, delim_whitespace=True)
 return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
 loaded = list()
 for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
 # stack group so that features are the 3rd dimension
 loaded = dstack(loaded)
 return loaded

# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
 filepath = prefix + group + '/Inertial Signals/'
 # load all 9 files as a single array
 filenames = list()
 # total acceleration
 filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
 # body acceleration
 filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
 # body gyroscope
 filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
 # load input data
 X = load_group(filenames, filepath)
 # load class output
 y = load_file(prefix + group + '/y_'+group+'.txt')
 return X, y

# load all train
trainX, trainy = load_dataset('train', '/content/drive/MyDrive/UCI HAR Dataset/')
print(trainX.shape, trainy.shape)
# load all test
testX, testy = load_dataset('test', '/content/drive/MyDrive/UCI HAR Dataset/')
print(testX.shape, testy.shape)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)


In [ ]:
# summarize the balance of classes in an output variable column
def class_breakdown(data):
 # convert the numpy array into a dataframe
 df = DataFrame(data)
 # group data by the class value and calculate the number of rows
 counts = df.groupby(0).size()
 # retrieve raw rows
 counts = counts.values
 # summarize
 for i in range(len(counts)):
     percent = counts[i] / len(df) * 100
     print('Class=%d, total=%d, percentage=%.3f' % (i+1, counts[i], percent))

In [ ]:

# summarize class balance
from numpy import array
from numpy import vstack
from pandas import read_csv
from pandas import DataFrame

# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# summarize the balance of classes in an output variable column
def class_breakdown(data):
 # convert the numpy array into a dataframe
 df = DataFrame(data)
 # group data by the class value and calculate the number of rows
 counts = df.groupby(0).size()
 # retrieve raw rows
 counts = counts.values
 # summarize
 for i in range(len(counts)):
      percent = counts[i] / len(df) * 100
      print('Class=%d, total=%d, percentage=%.3f' % (i+1, counts[i], percent))

# load train file
trainy = load_file('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt')
# summarize class breakdown
print('Train Dataset')
class_breakdown(trainy)

# load test file
testy = load_file('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt')
# summarize class breakdown
print('Test Dataset')
class_breakdown(testy)

# summarize combined class breakdown
print('Both')
combined = vstack((trainy, testy))
class_breakdown(combined)

Train Dataset
Class=1, total=1226, percentage=16.676
Class=2, total=1073, percentage=14.595
Class=3, total=986, percentage=13.411
Class=4, total=1286, percentage=17.492
Class=5, total=1374, percentage=18.689
Class=6, total=1407, percentage=19.138
Test Dataset
Class=1, total=496, percentage=16.831
Class=2, total=471, percentage=15.982
Class=3, total=420, percentage=14.252
Class=4, total=491, percentage=16.661
Class=5, total=532, percentage=18.052
Class=6, total=537, percentage=18.222
Both
Class=1, total=1722, percentage=16.720
Class=2, total=1544, percentage=14.992
Class=3, total=1406, percentage=13.652
Class=4, total=1777, percentage=17.254
Class=5, total=1906, percentage=18.507
Class=6, total=1944, percentage=18.876


In [ ]:
# load data
trainX, trainy = load_dataset('train', '/content/drive/MyDrive/UCI HAR Dataset/')

In [ ]:
sub_map = load_file('/content/drive/MyDrive/UCI HAR Dataset/train/subject_train.txt')
train_subjects = np.unique(sub_map)
print(train_subjects)

[ 1  3  5  6  7  8 11 14 15 16 17 19 21 22 23 25 26 27 28 29 30]


In [ ]:
# get all data for one subject
def data_for_subject(X, y, sub_map, sub_id):
 # get row indexes for the subject id
 ix = [i for i in range(len(sub_map)) if sub_map[i]==sub_id]
 # return the selected samples
 return X[ix, :, :], y[ix]

In [ ]:
# convert a series of windows to a 1D list
def to_series(windows):
 series = list()
 for window in windows:
 # remove the overlap from the window
  half = int(len(window) / 2) - 1
 for value in window[-half:]:
  series.append(value)
 return series

In [ ]:
# plot the data for one subject
def plot_subject(X, y):
 plt.figure()
 # determine the total number of plots
 n, off = X.shape[2] + 1, 0
 # plot total acc
 for i in range(3):
  plt.subplot(n, 1, off+1)
 plt.plot(to_series(X[:, :, off]))
 plt.title('total acc '+str(i), y=0, loc='left')
 off += 1
 # plot body acc
 for i in range(3):
  plt.subplot(n, 1, off+1)
 plt.plot(to_series(X[:, :, off]))
 plt.title('body acc '+str(i), y=0, loc='left')
 off += 1
 # plot body gyro
 for i in range(3):
  plt.subplot(n, 1, off+1)
 plt.plot(to_series(X[:, :, off]))
 plt.title('body gyro '+str(i), y=0, loc='left')
 off += 1
 # plot activities
 plt.subplot(n, 1, n)
 plt.plot(y)
 plt.title('activity', y=0, loc='left')
 plt.show()

In [ ]:
# load all train
X_train, Y_train = load_dataset_group('train', '/content/drive/MyDrive/UCI HAR Dataset/')
# load all test
X_test, Y_test = load_dataset_group('test', '/content/drive/MyDrive/UCI HAR Dataset/')
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
# one hot encode y
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)

File Path :  /content/drive/MyDrive/UCI HAR Dataset/train/Inertial Signals/
File Path :  /content/drive/MyDrive/UCI HAR Dataset/test/Inertial Signals/
X_train.shape :  (7352, 128, 1)
Y_train.shape :  (7352, 6)
X_test.shape :  (2947, 128, 1)
Y_test.shape :  (2947, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_acc', verbose=1 , save_best_only=True, save_weights_only=False, mode='auto')

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
 verbose, epochs, batch_size = 0, 10, 32
 n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
 model = Sequential()
 model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
 model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
 model.add(Dropout(0.5))
 model.add(MaxPooling1D(pool_size=2))
 model.add(Flatten())
 model.add(Dense(100, activation='relu'))
 model.add(Dense(n_outputs, activation='softmax'))
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit network
 model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
 # evaluate model
 _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
 return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
 print(scores)
 m, s = mean(scores), std(scores)
 print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=10):
 # load data
 trainX, trainy, testX, testy = pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt"),pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt"),pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt"),pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt")
 # repeat experiment
 scores = list()
 for r in range(repeats):
  score = evaluate_model(trainX, trainy, testX, testy)
 score = score * 100.0
 print('>#%d: %.3f' % (r+1, score))
 scores.append(score)
 # summarize results
 summarize_results(scores)

In [ ]:
run_experiment()